---


# merge

### 위의 시트에서 사본시트에서 코드를 확인하여 일치하는 보험 청구 항목을 새로운 열로 추가하는 코드입니다

In [ ]:
import gspread
import re
import time
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

doc = gc.open_by_url(spreadsheet_url)

insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 추가된 열의 이름 저장
disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])

for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    if not disease_code.strip():
        clean_category = ""
        clean_item = ""
    elif "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(",")

            if disease_code in [code.strip() for code in claim_codes]:
                category = claim_row[0]
                item = claim_row[1]
                break

        clean_category = re.sub(r"[0-9.]", "", category).strip()
        clean_item = re.sub(r"[0-9.]", "", item).strip()

    disease_code_sheet.update(
        f"C{disease_row_idx}:D{disease_row_idx}", [[clean_category, clean_item]]
    )
    time.sleep(2)

In [1]:
import gspread
import re
import time
from oauth2client.service_account import ServiceAccountCredentials

scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
creds = ServiceAccountCredentials.from_json_keyfile_name("key.json", scope)
gc = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1PzFuHuF2DvMPdSI-TK4Kq5LjPwyGzIxH3Q0Drnqft20/edit?usp=sharing"

doc = gc.open_by_url(spreadsheet_url)

insurance_claim_sheet = doc.worksheet("행위별 Category")
disease_code_sheet = doc.worksheet("상병 Category의 사본")

insurance_claim_data = insurance_claim_sheet.get_all_values()
disease_code_data = disease_code_sheet.get_all_values()

# 추가된 열의 이름 저장
disease_code_sheet.update("C1:D1", [["청구 카테고리", "세부 청구 항목"]])

# 상병 코드를 키로 하여 중복된 코드를 저장할 수 있는 딕셔너리 생성
disease_code_tracker = {}

for disease_row_idx, disease_row in enumerate(disease_code_data[1:], start=2):
    disease_code = disease_row[0]

    # 상병 코드가 비어있거나 '~'를 포함하면 무시
    if not disease_code.strip() or "~" in disease_code:
        clean_category = ""
        clean_item = ""
    else:
        # 기존 항목이 없을 경우 초기값 설정
        category = "일치하는 보험 상품이 없습니다"
        item = "일치하는 보험 상품이 없습니다"

        for claim_row in insurance_claim_data[1:]:
            claim_codes = claim_row[3].split(",")

            if disease_code in [code.strip() for code in claim_codes]:
                category = claim_row[0]
                item = claim_row[1]

                # 중복된 상병 코드를 찾을 때마다 새로운 행에 추가
                clean_category = re.sub(r"[0-9.]", "", category).strip()
                clean_item = re.sub(r"[0-9.]", "", item).strip()

                # 상병 코드가 중복되는 경우, 기존 데이터에 행을 추가
                if disease_code in disease_code_tracker:
                    next_row = len(disease_code_data) + 1
                    disease_code_sheet.append_row(
                        [disease_code, "", clean_category, clean_item]
                    )
                else:
                    disease_code_tracker[disease_code] = (
                        True  # 이미 처리된 상병 코드로 기록
                    )
                    disease_code_sheet.update(
                        f"C{disease_row_idx}:D{disease_row_idx}",
                        [[clean_category, clean_item]],
                    )
                time.sleep(2)

FileNotFoundError: [Errno 2] No such file or directory: 'key.json'